<img src="uc3m-logo.jpg" alt="UC3M logo" width="200"/>

# Second Homework: Network Optimization and Non-linear Models (Topics 2, 3)
## BsC in Data Science and Engineering - Optimization and Analytics (2022-2023)

Author: Rodrigo Oliver Coimbra (NIA: 100451788)

## 1. Introduction

## 2. Network optimization (with discrete variables) [Pyomo/Gurobi]

### 2.0 Model explanation 

#### 2.0.0 General conditions (CPM/PERM)

#### 2.0.1 Specific conditions

### 2.1 Python implementation of network concepts

In [1]:
# We import the libraries that are needed
# for solving linear programming problems
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# For better data manipulation
# we import Numpy, the Numeric Python library
import numpy as np

# For performance analysis purposes
from time import process_time

# For saving the results returned by the solver
from pandas import Series, DataFrame

# For better code readability and maintanability
from typing import Final, List, Dict
from dataclasses import dataclass

In [2]:
class Activity:
    """
    This class defines the activities that make up the project to be crashed (optimized),
    each activity has a name (to make it more human-readable), a start event, an end event,
    a normal duration, a crashed duration, the normal cost that it represents and the cost
    that crashing would entail.
    """
    
    def __init__(self, name: str, start: int, end: int, normal_time: int, crash_time: int, normal_cost: int, crash_cost: int):
        self.name = name
        self.start = start
        self.end = end
        self.normal_time = normal_time
        self.crash_time = crash_time
        self.normal_cost = normal_cost
        self.crash_cost = crash_cost
        
        if normal_cost == 0 or crash_time == 0:
            self.C = 0
        else:
            self.C = int((crash_cost - normal_cost)/(normal_time - crash_time))

def generateA (A: np.ndarray, start_nodes: List[int], end_nodes: List[int]) -> None:
    """
    This function takes an empty matrix A, a list of 
    starting nodes and a list of ending ones; it converts
    it into an incidence matrix of the activities (arcs)
    and events (nodes)
    """
    
    count = 0
    for i in zip(start_nodes,end_nodes):
        A[i[0]][count] = 1
        A[i[1]][count] = -1
        count+=1
    
    return

def startNodes(alist: List[int]) -> List[int]:
    """Returns the list of starting events (nodes)"""
    return [(a.start-1) for a in alist]

def endNodes(alist: List[int]) -> List[int]:
    """Returns the list of ending events (nodes)"""
    return [(a.end-1) for a in alist]

def diffTime(alist: List[int]) -> List[int]:
    """Returns the time difference between
    the normal duration and the crash one"""
    return [(a.normal_time - a.crash_time) for a in alist]

def cValues(alist: List[int]) -> List[int]:
    """Returns the cost-to-time ratio"""
    return [a.C for a in alist]

def normalTime(alist: List[int]) -> List[int]: 
    """Returns the normal time duration by activity"""
    return [a.normal_time for a in alist]

def networkDict(alist: List[int], nodes: int, arcs: int, M: np.ndarray) -> Dict:
    """
    Returns a dictionary containing the following information:
    (i,j):(a,b) where (i,j) are the rows and columns of the incidence
    matrix - respectively - and (a,b) represent the start and end
    nodes of a give activity.
    """
    
    temp = {}
    res = {}
    for i, e in enumerate(alist):
        temp[i] = (e.start-1, e.end-1)
    for i in range(nodes):
        for j in range(arcs):
            if M[i][j] == -1:
                res[(i,j)] = temp[j]
    return res

def uniqueEndNodes(alist: List[int]) -> int:
    """Returns a list of unique end nodes"""
    return len(set(endNodes(alist)))

def numberNodes(alist: List[int]) -> int:
    """Returns the number of unique nodes in the model"""
    return len(set(endNodes(alist))) + 1

### 2.2 Data creation and manipulation

In [3]:
# We create the activity arcs between the
# event nodes with all the necessary information
"""
activity_list = [
    Activity('A', 0, 1, 5, 2, 1_000, 1_500),
    Activity('B', 1, 2, 2, 1, 1_500, 2_000),
    Activity('C', 2, 3, 10, 3, 5_500, 7_000),
    Activity('D', 1, 4, 3, 1, 250, 500),
    Activity('E', 4, 5, 7, 2, 400, 650),
    Activity('F', 5, 6, 5, 1, 500, 750),
    Activity('G', 5, 7, 6, 2, 700, 800),
    Activity('H', 6, 8, 2, 1, 500, 900),
    Activity('I', 7, 9, 3, 1, 100, 300),
    Activity('J', 7, 10, 15, 4, 3_500, 5_500),
    Activity('K', 8, 11, 12, 3, 3_000, 4_750),
    Activity('L', 9, 11, 4, 1, 600, 800),
    Activity('M', 10, 11, 8, 3, 1_000, 1_400),
    Activity('N', 11, 12, 5, 2, 700, 1_000),
    Activity('O', 11, 13, 10, 1, 2_800, 3_100),
    Activity('P', 12, 14, 20, 3, 6_000, 6_900),
    Activity('Q', 13, 18, 5, 2, 700, 800),
    Activity('R', 13, 19, 2, 1, 300, 450),
    Activity('S', 14, 15, 3, 1, 200, 300),
    Activity('T', 14, 16, 15, 3, 4_000, 4_700),
    Activity('U', 18, 20, 21, 5, 6_000, 8_500),
    Activity('V', 19, 20, 30, 8, 10_500, 16_000),
    Activity('W', 15, 17, 10, 3, 3_800, 4_000),
    Activity('X', 17, 21, 3, 1, 200, 250),
    Activity('Y', 3, 15, 2, 1, 3_000, 3_500),
    Activity('Z', 16, 20, 5, 2, 1_500, 2_000),
    Activity('Dummy 1', 3, 5, 0, 0, 0, 0),
    Activity('Dummy 2', 20, 17, 0, 0, 0, 0)
]
"""
activity_list = [Activity('A', 1, 2, 3, 2, 16_000, 18_000),
               Activity('B', 2, 3, 5, 3, 20_000, 23_000),
               Activity('C', 2, 4, 3, 2, 4_000, 8_000),
               Activity('D', 4, 5, 1, 0.5, 1_000, 1_500),
               Activity('E', 3, 7, 3, 1, 12_000, 17_000),
               Activity('F', 5, 6, 4, 2, 20_000, 26_000),
               Activity('G', 4, 6, 2, 1, 1_000, 1_800),
               Activity('H', 6, 7, 3, 1, 1_200, 2_600),
               Activity('I', 7, 8, 2, 1.5, 1_000, 1_400),
               Activity('Dummy', 3, 5, 0, 0, 0, 0)
                ]


# Define three important constants:
# the number of events (nodes); the number of
# activities (arcs); and the deadline in days
# of the project
E: Final = numberNodes(activity_list)
A: Final = len(activity_list)
DEADLINE: Final = 13

# Check that E and A are equal
# to their theoretical value
#assert E == 22
#assert A == 28

# Obtain two lists: one with the 
# starting nodes and another one
# with the ending ones
start_nodes = startNodes(activity_list)
end_nodes = endNodes(activity_list)

# Create a E times A incidence matrix
# in this case it has 22*28 = 616 entries
M = np.zeros((E, len(end_nodes)))
generateA(M, start_nodes, end_nodes)

# Check that M has the correct shape
assert M.shape == (E, A)

# Create a dictionary with all the necessary 
# information to implement the network constraints
activity_dict = networkDict(activity_list, E, A, M)

# Check that number of elements in the dictionary
# correspond to the number of activities
assert len(activity_dict) == A

# Print the dictionary
print(f"{activity_dict=}")

activity_dict={(1, 0): (0, 1), (2, 1): (1, 2), (3, 2): (1, 3), (4, 3): (3, 4), (4, 9): (2, 4), (5, 5): (4, 5), (5, 6): (3, 5), (6, 4): (2, 6), (6, 7): (5, 6), (7, 8): (6, 7)}


### 2.3 Network Linear Programming implementation

#### 2.3.1 Problem data definition: sets and parameters

In [4]:
# We create the Pyomo model to be used
network_model = pyo.ConcreteModel()

# Create two lists: one for the event labels
# and another one for the activities labels
# to be used a Sets for the model
event_labels = [e for e in range(E)]
activity_labels = [a for a in range(A)]

# Initialize both of these lists to the model,
# with the events corresponding to the i dimension
# and the activities corresponding to the j dimension
network_model.i = pyo.Set(initialize=event_labels)
network_model.j = pyo.Set(initialize=activity_labels)

# Fetch the c values for all activities in the model
c = cValues(activity_list)
# Initialize these values into the model as parameters
network_model.costs = pyo.Param(network_model.j, initialize=c, doc="Cost in monetary units per activity per time unit")

# Fetch the normal time values for all activities in the model
normal_time = normalTime(activity_list)
# Initialize these values into the model as parameters
network_model.normaltime = pyo.Param(network_model.j, initialize=normal_time, doc="Normal time in days for the completion of a given activity")

# Fetch the difference between normal and crashed times
# for all activities in the model
diff_time = diffTime(activity_list)
# Initialize these values into the model as parameters
network_model.timediff = pyo.Param(network_model.j, initialize=diff_time, doc="Time difference in days between the normal time and the crash time")

#### 2.3.2 Define the variables

In [5]:
# Define and add to the model the x and y variables,
# living in the i and j dimensions, respectively
network_model.x = pyo.Var(network_model.i, within=pyo.NonNegativeReals)
network_model.y = pyo.Var(network_model.j, within=pyo.NonNegativeReals)

# Print the variables that were generated
network_model.x.pprint()
network_model.y.pprint()

x : Size=8, Index=i
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  None :  None : False :  True : NonNegativeReals
      1 :     0 :  None :  None : False :  True : NonNegativeReals
      2 :     0 :  None :  None : False :  True : NonNegativeReals
      3 :     0 :  None :  None : False :  True : NonNegativeReals
      4 :     0 :  None :  None : False :  True : NonNegativeReals
      5 :     0 :  None :  None : False :  True : NonNegativeReals
      6 :     0 :  None :  None : False :  True : NonNegativeReals
      7 :     0 :  None :  None : False :  True : NonNegativeReals
y : Size=10, Index=j
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  None :  None : False :  True : NonNegativeReals
      1 :     0 :  None :  None : False :  True : NonNegativeReals
      2 :     0 :  None :  None : False :  True : NonNegativeReals
      3 :     0 :  None :  None : False :  True : NonNegativeReals
      4 :     0 :  None :  None : False :

#### 2.3.3 Define the objective function

In [6]:
def n_objective(n_model: pyo.ConcreteModel) -> bool:
    """
    The objective function is given by the minimization of the sum
    of the unit crashing cost times the crashing time units over 
    all the model activities
    """
    
    return sum(n_model.costs[j]*n_model.y[j] for j in n_model.j)

# Add the objective function rule into the model
# Even though Pyomo by default performs minimization
# it is marked explicitly here
network_model.objective = pyo.Objective(rule=n_objective, sense=pyo.minimize)


# Print the objective function
network_model.objective.pprint()

objective : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 2000*y[0] + 1500*y[1] + 4000*y[2] + 1000*y[3] + 2500*y[4] + 3000*y[5] + 800*y[6] + 700*y[7] + 800*y[8]


#### 2.3.4 Define the constraints

In [7]:
##########################
# CRASH TIME CONSTRAINTS #
##########################

def crash_time_constraints(n_model: pyo.ConcreteModel, j: int) -> bool:
    """Crash time constraints establish that the crash time
    for a given activity j must be lesser or equal to the difference
    between the normal and crashed times for that activity
    """
    
    return n_model.y[j] <= n_model.timediff[j]

# Add the crash time constraint into the model
network_model.crash_time = pyo.Constraint(network_model.j, rule=crash_time_constraints)
# Print the crash time constraints
network_model.crash_time.pprint()


crash_time : Size=10, Index=j, Active=True
    Key : Lower : Body : Upper : Active
      0 :  -Inf : y[0] :   1.0 :   True
      1 :  -Inf : y[1] :   2.0 :   True
      2 :  -Inf : y[2] :   1.0 :   True
      3 :  -Inf : y[3] :   0.5 :   True
      4 :  -Inf : y[4] :   2.0 :   True
      5 :  -Inf : y[5] :   2.0 :   True
      6 :  -Inf : y[6] :   1.0 :   True
      7 :  -Inf : y[7] :   2.0 :   True
      8 :  -Inf : y[8] :   0.5 :   True
      9 :  -Inf : y[9] :   0.0 :   True


In [8]:
#######################
# NETWORK CONSTRAINTS #
#######################

def network_limit_constraints(n_model: pyo.ConcreteModel, a_dict: dict, constraintList: pyo.ConstraintList) -> None:
    """
    Network limit constraints establish that the time where a certain event i (node i)
    is reached (with respect with the initial even time of 0) is always greater or equal to
    the result of the normal duration of activity j that is related with the occurrence of 
    the event, minus the crashing time of the said activity j plus the occurrence time of
    the predecessor event k (node k) where the activity j originates from
    """
    # Rearanging the terms of the expression described above and using the
    # information provided by the problem dictionary
    for k, v in a_dict.items():
        constraintList.add(expr = n_model.x[v[1]] + n_model.y[k[1]] - n_model.x[v[0]] >= n_model.normaltime[k[1]])
    constraintList.pprint()
    return

In [9]:
# Create the constraint list
network_model.network_constraints = pyo.ConstraintList()
# Add the network inherent constraint list to the model
network_limit_constraints(network_model, activity_dict, network_model.network_constraints)

network_constraints : Size=10, Index=network_constraints_index, Active=True
    Key : Lower : Body               : Upper : Active
      1 :   3.0 : x[1] + y[0] - x[0] :  +Inf :   True
      2 :   5.0 : x[2] + y[1] - x[1] :  +Inf :   True
      3 :   3.0 : x[3] + y[2] - x[1] :  +Inf :   True
      4 :   1.0 : x[4] + y[3] - x[3] :  +Inf :   True
      5 :   0.0 : x[4] + y[9] - x[2] :  +Inf :   True
      6 :   4.0 : x[5] + y[5] - x[4] :  +Inf :   True
      7 :   2.0 : x[5] + y[6] - x[3] :  +Inf :   True
      8 :   3.0 : x[6] + y[4] - x[2] :  +Inf :   True
      9 :   3.0 : x[6] + y[7] - x[5] :  +Inf :   True
     10 :   2.0 : x[7] + y[8] - x[6] :  +Inf :   True


In [10]:
##################################
# PROJECT COMPLETION CONSTRAINTS #
##################################
def project_completion_constraints(n_model: pyo.ConcreteModel, a_dict: dict, constraintList: pyo.ConstraintList) -> None:
    """
    Project completion constraints establish that the time occurence of the initial event
    is equal to 0 and that the conclusion date of the project must be lesser or equal
    than the deadline that is specified
    """
    # The start of the project is at time t=0
    constraintList.add(expr = n_model.x[0] == 0)
    # The project must have been completed by the deadline that is specified
    constraintList.add(expr = n_model.x[len(n_model.x)-1] <= DEADLINE)
    # We print these constraint after being added to the model
    constraintList.pprint()
    return

In [11]:
# We define the list for project completion constraints
network_model.completion_constraints = pyo.ConstraintList()

# We call the function project completion constraints
# and add these constraints to the  model
project_completion_constraints(network_model, activity_dict, network_model.completion_constraints)

completion_constraints : Size=2, Index=completion_constraints_index, Active=True
    Key : Lower : Body : Upper : Active
      1 :   0.0 : x[0] :   0.0 :   True
      2 :  -Inf : x[7] :  13.0 :   True


#### 2.3.5 Solve the linear programming model

In [12]:
# We import using pyo.Suffix to later
# perform a sensitivities analysis
network_model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# We use the LP Gurobi solver
# to find the optimal solution
Solver = SolverFactory('gurobi')
Results = Solver.solve(network_model)

#### 2.3.6 Interpret the network linear programming model results

In [13]:
# Display the solution for problem
network_model.display()

Model unknown

  Variables:
    x : Size=8, Index=i
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :  None : False : False : NonNegativeReals
          1 :     0 :   2.5 :  None : False : False : NonNegativeReals
          2 :     0 :   6.5 :  None : False : False : NonNegativeReals
          3 :     0 :   5.5 :  None : False : False : NonNegativeReals
          4 :     0 :   6.5 :  None : False : False : NonNegativeReals
          5 :     0 :  10.5 :  None : False : False : NonNegativeReals
          6 :     0 :  11.5 :  None : False : False : NonNegativeReals
          7 :     0 :  13.0 :  None : False : False : NonNegativeReals
    y : Size=10, Index=j
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.5 :  None : False : False : NonNegativeReals
          1 :     0 :   1.0 :  None : False : False : NonNegativeReals
          2 :     0 :   0.0 :  None : False : False : NonNegativeReals
          3 :     0 

#### 2.3.7 Sensitivies and their interpretation